# 2. Renaming and merging

Note: Half datasets, with separate files for east and west subplots have been merged manually in excel

In [1]:
%%time

import os
import math
import datetime
import numpy as np
import pandas as pd
from copy import copy

# Dictionaries
import json
from pprint import pprint

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# To display df nicely in loops
from IPython.display import display 
# display(df1.head()) 
# display(df2.head())

# Display rows and columns Pandas
pd.options.display.max_columns = 100
pd.set_option('display.max_rows',100)

Wall time: 1.16 s


In [2]:
# Prints the current working directory
os.getcwd()
# os.listdir()

'C:\\Users\\fahad\\MegaSync\\NMBU\\GitHub\\vPheno'

## Finding Username folder to make general path for multi PC use

In [3]:
username = str(os.getcwd()).split('\\')[2]
username

'fahad'

## Importing Data

In [4]:
path = r'./Data/'
path_complete = r'./Data/complete/'
path_incomplete = r'./Data/incomplete/'

path_yield = r'C:\\Users\\'+username+'\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021'

export_path = './Data/renamed_merged/'

# Create export_path folder if not exists already
os.makedirs(export_path, exist_ok=True)

os.listdir(path)

['complete',
 'incomplete',
 'renamed_merged',
 'results',
 'std_columns.json',
 'yield_df.json']

## Data Preparation
### Creating list of complete files

In [5]:
# Get the list of all files in directory tree at given path

files_with_address = []
files_list = []

for (dirpath, dirnames, filenames) in os.walk(path_complete):
    files_with_address += [os.path.join(dirpath, file) for file in filenames]
    files_list.extend(filenames)
    
print(len(files_with_address), 'files found in the directory')
# files_with_address
# files_list

63 files found in the directory


## Data Checking/control

### Check for duplicate filenames

In [6]:
print('Total number of files are :', len(files_list))

print('Number of unique file names are:', len(set(files_list)))

print('There is/are', len(files_list) - len(set(files_list)),'duplicate file name/names.')
if len(files_list) - len(set(files_list)) > 0:
    raise NameError

Total number of files are : 63
Number of unique file names are: 63
There is/are 0 duplicate file name/names.


In [7]:
# files_list

In [8]:
len(files_with_address)
# files_with_address

63

## Importing data files to Pandas

In [9]:
%%time

all_df = []
for data in files_with_address:
    file_name = os.path.splitext(os.path.basename(data))[0]

    # Replce all invalid characters in the name
    file_name = file_name.replace(" ", "_")
    file_name = file_name.replace("-", "_")
    file_name = file_name.replace(")", "")
    file_name = file_name.replace("(", "")
    df_name = file_name.replace(".", "")
    # Test: Check if the same date is already present in the current dict key
    if df_name in all_df:
        print(f'A file with the same name {df_name} has already been imported. \n Please check if there is duplication of data.')
        raise NameError
    all_df.append(df_name)

    locals()[df_name] = pd.read_csv(data, index_col=False)
    print(df_name, '=====', locals()[df_name].shape)

Graminor_050719_plots_826,_837_deleted,_one_missing_row_deleted ===== (597, 49)
Graminor_060619 ===== (600, 49)
Graminor_070819_2 ===== (600, 49)
Graminor_110619 ===== (600, 49)
Graminor_150719 ===== (600, 49)
Graminor_150819 ===== (600, 49)
Graminor_250719 ===== (600, 46)
Graminor_280619_corrected ===== (600, 49)
Graminor_eastwest_020719_NIR_half_missing ===== (600, 22)
Graminor_eastwest_040720 ===== (800, 49)
Graminor_eastwest_040820 ===== (793, 22)
Graminor_eastwest_050819 ===== (600, 49)
Graminor_eastwest_070720_correct ===== (800, 49)
Graminor_eastwest_130720 ===== (800, 22)
Graminor_eastwest_140820 ===== (800, 49)
Graminor_eastwest_300720 ===== (787, 22)
Graminor_east_010720 ===== (400, 51)
Graminor_east_110719 ===== (300, 49)
Graminor_east_170720cpg ===== (400, 49)
Graminor_east_180620 ===== (400, 46)
Graminor_east_200720 ===== (400, 22)
Graminor_Mica_eastcorrect_west_240620 ===== (757, 49)
Masbasis_050719_corrected ===== (528, 49)
Masbasis_060619_Indices ===== (528, 56)
Masbasi

In [10]:
print(f'Total imported {len(all_df)}')

Total imported 63


# Drop derieved indices

## Import and process standard columns json

In [11]:
std_cols_json = open("Data\std_columns.json", "r")
import_columns = std_cols_json.read()
std_cols_json.close()
type(import_columns)

str

In [12]:
# Converting list stored as string to list
import_columns = import_columns.strip('][').split(', ')

# Stripping extra quotes enclosing the strings
x=0
std_columns = []
for item in import_columns: 
    std_columns.append(item[1:-1])
    x=+1
std_columns

['Plot_ID',
 'Blue_Mean_Value',
 'Blue_Median_Value',
 'Blue_Std_Dev_Value',
 'Green_Mean_Value',
 'Green_Median_Value',
 'Green_Std_Dev_Value',
 'Red_Mean_Value',
 'Red_Median_Value',
 'Red_Std_Dev_Value',
 'RedEdge_Mean_Value',
 'RedEdge_Median_Value',
 'RedEdge_Std_Dev_Value',
 'NIR_Mean_Value',
 'NIR_Median_Value',
 'NIR_Std_Dev_Value']

In [13]:
# Creating lists of standard columns for Mean, Median and Std_Dev

std_columns_mean = ['Plot_ID']
std_columns_median = ['Plot_ID']
std_columns_stdev = ['Plot_ID']

for item in std_columns:
    if item.find("Mean") != -1:
        std_columns_mean.append(item)
    if item.find("Median") != -1:
        std_columns_median.append(item)
    if item.find("Std_Dev") != -1:
        std_columns_stdev.append(item)
print(std_columns_mean)
print(std_columns_median)
print(std_columns_stdev)

['Plot_ID', 'Blue_Mean_Value', 'Green_Mean_Value', 'Red_Mean_Value', 'RedEdge_Mean_Value', 'NIR_Mean_Value']
['Plot_ID', 'Blue_Median_Value', 'Green_Median_Value', 'Red_Median_Value', 'RedEdge_Median_Value', 'NIR_Median_Value']
['Plot_ID', 'Blue_Std_Dev_Value', 'Green_Std_Dev_Value', 'Red_Std_Dev_Value', 'RedEdge_Std_Dev_Value', 'NIR_Std_Dev_Value']


## Dropping extra/derieved indices columns from dataframes

In [14]:
# Filtering the datasets with only the standard columns

for df in all_df:
    locals()[df] = locals()[df].copy()[std_columns]

# ToDo: Dropping NAN

## Finding NAN values
### ToDo: Test: Raise error if missing values found

In [15]:
# Finding number of missing values in each dataframe
df_with_nan = []
missing_values = False
for df in all_df:
    if locals()[df].isna().sum().sum() > 0:
        print(f'Total missing values in {df} are {locals()[df].isna().sum().sum()}')
        missing_values = True
        df_with_nan.append(df)
#     if len(df_with_nan) > 0:
#         raise ValueError
if not missing_values:
    print('No missing value found in any dataframe')

Total missing values in Graminor_eastwest_020719_NIR_half_missing are 900


In [16]:
df_with_nan

['Graminor_eastwest_020719_NIR_half_missing']

In [17]:
# Finding which column has NAN values
for df in df_with_nan:
    print(f'{df}:\n {locals()[df].shape[1]-locals()[df].dropna(axis=1).shape[1]} columns or {locals()[df].shape[0]-locals()[df].dropna().shape[0]} rows to be dropped,')

Graminor_eastwest_020719_NIR_half_missing:
 3 columns or 300 rows to be dropped,


## ToDo: Automate: Drop rows with missing values in df_with_nan

In [18]:
for df in df_with_nan:
    print(f'{locals()[df].shape} Before dropping')
    locals()[df].dropna(inplace=True)
    print(f'{locals()[df].shape} After dropping')


(600, 16) Before dropping
(300, 16) After dropping


## ORRR

## ToDo: Droppping df with Nan from the all_df_std

In [19]:
print(f'Number of items in all_df is {len(all_df)}')

Number of items in all_df is 63


In [20]:
# for df in df_with_nan:
#     all_df.remove(df)

###  ToDo: Update field_year_dict and sorted_field_year_dict after dropping the dataset

In [21]:
print(f'Number of items in all_df now is {len(all_df)}')

Number of items in all_df now is 63


# Standardizing the names

## Creating a dictionary with all dates for a certain field for a certain year

In [22]:
len(all_df)
# all_df

63

In [23]:
%%time

elements_to_strip = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ_-()."

all_df_std = []

field_year_dict = {}

# A reference dict to keep record of the names of files before they got renamed 
renamed_to_from = {}

for df in all_df:
    
    # Getting date from the df name
    date1 = copy(df)

    for x in range(3):
        date1 = date1.rstrip(elements_to_strip)
        date1 = date1.lstrip(elements_to_strip)
        for c in range(3):
            date1 = date1.rstrip(elements_to_strip)
            date1 = date1.lstrip(elements_to_strip)
        date1 = date1.split('_')[0]
    
    field_name = df.split('_')[0]
    field_name = field_name.split('-')[0]

    new_df_name = field_name +'_'+date1
    
    # Drop all columns except the std columns 
    locals()[new_df_name] = locals()[df][std_columns]
    
    all_df_std.append(new_df_name)

    
    
    # Creating a dict with all dates for a certain field for a certain year
    dict_key = field_name+'_20'+date1[-2:]
    
    if dict_key in field_year_dict:
        # Test: Check if the same date is already present in the current dict key
        if date1 in field_year_dict[dict_key]:
            print(f'Duplicate Data file Error: {date1} is already present in {dict_key}\n Current df {df}\n Conflict with {renamed_to_from[new_df_name]}')
            raise NameError
        field_year_dict[dict_key].append(date1)
    else:
        field_year_dict[dict_key] = [date1]
    
    # Adding new and old names to a dict for record
    renamed_to_from[new_df_name] = df
    
#     print(date1)
print('field_year_dict created.')

pprint(field_year_dict)
# all_df_std


field_year_dict created.
{'Graminor_2019': ['050719',
                   '060619',
                   '070819',
                   '110619',
                   '150719',
                   '150819',
                   '250719',
                   '280619',
                   '020719',
                   '050819',
                   '110719'],
 'Graminor_2020': ['040720',
                   '040820',
                   '070720',
                   '130720',
                   '140820',
                   '300720',
                   '010720',
                   '170720',
                   '180620',
                   '200720',
                   '240620'],
 'Masbasis_2019': ['050719',
                   '060619',
                   '070819',
                   '150719',
                   '220719',
                   '260619',
                   '290719',
                   '280619'],
 'Masbasis_2020': ['010720',
                   '070820',
                   '080720',
               

## Test: Check if there are duplicate datasets/names in all_df_std

In [24]:

if len(all_df_std) > len(set(all_df_std)):
    duplicates = len(all_df_std) - len(set(all_df_std))
    if duplicates>1:
        verb, plural='are', 's'
    else:
        verb, plural='is', ''
    print(f'Error:\nThere {verb} {duplicates} duplicate name{plural} in the datasets out of total {len(all_df_std)}.\n \
    Make sure no dataset has been lost because of data being separated in east/west fields on the same date')
    # Printing the names of the duplicate datasets, if any
    find_duplicates=[]
    for i in all_df_std:
        if i not in find_duplicates:
            find_duplicates.append(i)
        else:
            print(f'Duplicate dataset named \'{i}\',\n')
    raise NameError
    
else:
    print('No duplicate dataset found')

No duplicate dataset found


### Arranging the dates in field_year_Dict in ascending order


In [25]:
sorted_field_year_dict = {}
for key, dates_list in field_year_dict.items():
    # Converting the dates to a datetime date object and sorting them in list
    sorted_dated = sorted([
        datetime.datetime.strptime(date, '%d%m%y').date()
        for date in dates_list])

    sorted_field_year_dict[key] = sorted_dated

pprint(sorted_field_year_dict)

{'Graminor_2019': [datetime.date(2019, 6, 6),
                   datetime.date(2019, 6, 11),
                   datetime.date(2019, 6, 28),
                   datetime.date(2019, 7, 2),
                   datetime.date(2019, 7, 5),
                   datetime.date(2019, 7, 11),
                   datetime.date(2019, 7, 15),
                   datetime.date(2019, 7, 25),
                   datetime.date(2019, 8, 5),
                   datetime.date(2019, 8, 7),
                   datetime.date(2019, 8, 15)],
 'Graminor_2020': [datetime.date(2020, 6, 18),
                   datetime.date(2020, 6, 24),
                   datetime.date(2020, 7, 1),
                   datetime.date(2020, 7, 4),
                   datetime.date(2020, 7, 7),
                   datetime.date(2020, 7, 13),
                   datetime.date(2020, 7, 17),
                   datetime.date(2020, 7, 20),
                   datetime.date(2020, 7, 30),
                   datetime.date(2020, 8, 4),
                   da

# Identify & drop duplicate sub-plots in each datasets, if any

## Checking if duplicates exist

In [26]:
found_duplicates = False
for df in all_df_std:
    duplicates = locals()[df][locals()[df].duplicated(subset='Plot_ID')]['Plot_ID']
    if duplicates.size > 0:
        found_duplicates = True
        print(f'Following {duplicates.size} duplicates in {df} ')
        for x in range(duplicates.size):
            print(duplicates.iloc[x])
if found_duplicates:
    raise NameError
else:
    print(f'No duplicate subplots found in any dataset.')

No duplicate subplots found in any dataset.


## Finding out which plots are duplicate/ non-unique

In [27]:
found_duplicates = False

for df in all_df_std:
    plot_list_series = locals()[df].Plot_ID
    if len(plot_list_series[plot_list_series.duplicated()]) > 0:
        found_duplicates = True
        print(f'Duplicate subplots in {df} are {plot_list_series[plot_list_series.duplicated()]}')
        print(f'Number of plots {len(plot_list_series)}, vs Number of duplicate plots {len(set(plot_list_series))-len(plot_list_series)}')

if found_duplicates:
    raise NameError
else:
    print(f'No duplicate subplots found in any dataset.')

No duplicate subplots found in any dataset.


In [28]:
print(f'Number of plots {len(plot_list_series)}, vs Number of duplicate plots {len(set(plot_list_series))-len(plot_list_series)}')
print(Masbasis_130720.iloc[379,:])
print(Masbasis_130720.iloc[293,:])
Masbasis_130720.iloc[379,:] == Masbasis_130720.iloc[293,:]


Number of plots 96, vs Number of duplicate plots 0
Plot_ID                  1827.000000
Blue_Mean_Value             0.004315
Blue_Median_Value           0.004155
Blue_Std_Dev_Value          0.001230
Green_Mean_Value            0.010959
Green_Median_Value          0.010581
Green_Std_Dev_Value         0.003384
Red_Mean_Value              0.006048
Red_Median_Value            0.005484
Red_Std_Dev_Value           0.002242
RedEdge_Mean_Value          0.027218
RedEdge_Median_Value        0.026672
RedEdge_Std_Dev_Value       0.006525
NIR_Mean_Value              0.097022
NIR_Median_Value            0.097314
NIR_Std_Dev_Value           0.015057
Name: 379, dtype: float64
Plot_ID                  1727.000000
Blue_Mean_Value             0.004014
Blue_Median_Value           0.003858
Blue_Std_Dev_Value          0.000914
Green_Mean_Value            0.009829
Green_Median_Value          0.009518
Green_Std_Dev_Value         0.002195
Red_Mean_Value              0.004742
Red_Median_Value            0.00460

Plot_ID                  False
Blue_Mean_Value          False
Blue_Median_Value        False
Blue_Std_Dev_Value       False
Green_Mean_Value         False
Green_Median_Value       False
Green_Std_Dev_Value      False
Red_Mean_Value           False
Red_Median_Value         False
Red_Std_Dev_Value        False
RedEdge_Mean_Value       False
RedEdge_Median_Value     False
RedEdge_Std_Dev_Value    False
NIR_Mean_Value           False
NIR_Median_Value         False
NIR_Std_Dev_Value        False
dtype: bool

In [29]:
plots_list = (np.array(Masbasis_130720.Plot_ID.tolist()))

# ToDo Figure out the problem in the following code
seen = set(plots_list)
uniq = [x for x in plots_list if x not in seen and not seen.add(x)] 
uniq

[]

There are no duplicate datasets.  
There is a slight possibility that there may be duplicate sub-plots in a field. That is yet to be checked.


# Find grain yield, DH, DT etc values and attach to the dataframes

In [30]:
os.listdir(path_yield)

['2019 Staur Graminor',
 '2019 Staur Masbasis',
 '2019 Vollebekk Graminor',
 '2019 Vollebekk Masbasis',
 '2020 Staur Graminor',
 '2020 Staur Masbasis',
 '2020 Vollebekk Graminor',
 '2020 Vollebekk Masbasis',
 '2020 Vollebekk Robot',
 '2021 Vollebekk Masbasis']

## Data Preparation
### Creating list of all files

In [31]:
# Get the list of all files in directory tree at given path_yield

files_with_address = []
files_list = []

for (dirpath, dirnames, filenames) in os.walk(path_yield):
    files_with_address += [os.path.join(dirpath, file) for file in filenames]
    files_list.extend(filenames)
    
print(len(files_with_address), 'files found in the directory')
# files_with_address
# files_list
files_with_address_bkp = copy(files_with_address)

90 files found in the directory


### Checking if there are multiple sheets in the files

In [32]:
# Print number of sheets in all files
print('The following files have multiple sheets.')

list_multi_sheet = []
for file in files_with_address:
    xl_file = pd.ExcelFile(file,engine='openpyxl')
    number_of_sheets = len(xl_file.sheet_names)
    if number_of_sheets > 1:
        print(number_of_sheets, os.path.basename(file), 'in folder', os.path.basename(os.path.dirname(file))
)
        list_multi_sheet.append(file)

The following files have multiple sheets.
3 19BMLFN3 - MASBASIS yield trial Staur 2019.xlsx in folder 2019 Staur Masbasis
4 19TvPhenores.xlsx in folder 2019 Vollebekk Graminor
4 Graminor_Staur_Vollebekk_2019.xlsx in folder 2019 Vollebekk Graminor
4 19BMLGI1 - MASBASIS yield trial Vollebekk 2019.xlsx in folder 2019 Vollebekk Masbasis
3 Masbasis_vollebekk trial_data_2020.xlsx in folder 2019 Vollebekk Masbasis
3 Staur-Graminor-Masbasis_2020.xlsx in folder 2020 Staur Graminor
2 20BMLFN3 - MASBASIS avlingsforsøk Staur 2020 lodging data.xlsx in folder 2020 Staur Masbasis
2 Masbasis_2020_staur.xlsx in folder 2020 Staur Masbasis
2 Staur_maturity_heading_yield_2020.xlsx in folder 2020 Staur Masbasis
3 20BMLGI1_2020_tm.xlsx in folder 2020 Vollebekk Masbasis
3 Masbasis_Mica_2020_all_dates_MEDIAN_DP.xlsx in folder 2020 Vollebekk Masbasis
3 Masbasis_vollebekk trial_data_2020.xlsx in folder 2020 Vollebekk Masbasis
3 ROBOT_2020.xlsx in folder 2020 Vollebekk Robot


In [33]:
# Files dropped in the initial stage
list_multi_sheet

['C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Staur Masbasis\\19BMLFN3 - MASBASIS yield trial Staur 2019.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Vollebekk Graminor\\19TvPhenores.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Vollebekk Graminor\\Graminor_Staur_Vollebekk_2019.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Vollebekk Masbasis\\19BMLGI1 - MASBASIS yield trial Vollebekk 2019.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Vollebekk Masbasis\\Masbasis_vollebekk trial_data_2020.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2020 Staur Graminor\\Staur-Graminor-Masbasis_2020.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2020 Staur Masbasis\\20BMLFN3 - MASBASIS avlingsforsøk Staur 2020 lodging data.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Mas

### Identifying files without dates
(with 2019 in name means they dont have date format)

In [34]:
files_w_2019 = []
for file in files_with_address:
    if file not in list_multi_sheet:
        file_name = os.path.basename(file)
        if '2019' in file_name:
            print(file_name, 'in folder', os.path.basename(os.path.dirname(file)))
            files_w_2019.append(file)
        if '2020' in file_name:
            print(file_name, 'in folder', os.path.basename(os.path.dirname(file)))
            files_w_2019.append(file)

Staur_Graminor_2019_median 240719 070819 150819 210819 300819.xlsx in folder 2019 Staur Graminor
Staur_Masbasis_2019 240719 070819 150819 300819.xlsx in folder 2019 Staur Masbasis
Field_data_2019.xlsx in folder 2019 Vollebekk Masbasis
Masbasis_2019_ForDP_median.xlsx in folder 2019 Vollebekk Masbasis
NEW_Field_data_2019.xlsx in folder 2019 Vollebekk Masbasis
2020TGraminor-Vollebekk-res.xlsx in folder 2020 Vollebekk Graminor


In [35]:
files_w_2019

['C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Staur Graminor\\Staur_Graminor_2019_median 240719 070819 150819 210819 300819.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Staur Masbasis\\Staur_Masbasis_2019 240719 070819 150819 300819.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Vollebekk Masbasis\\Field_data_2019.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Vollebekk Masbasis\\Masbasis_2019_ForDP_median.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2019 Vollebekk Masbasis\\NEW_Field_data_2019.xlsx',
 'C:\\\\Users\\\\fahad\\MegaSync\\NMBU\\Master Thesis\\Data\\Feb2021\\2020 Vollebekk Graminor\\2020TGraminor-Vollebekk-res.xlsx']

In [36]:
len(all_df_std)

63

### Importing all files & their sheets into pandas

In [37]:
%%time

df_yield = []

# A reference dict to keep record of the names of files before they got renamed 
yield_import_to_from = {}

for data in list_multi_sheet+files_w_2019:
    file_name = os.path.splitext(os.path.basename(data))[0]
    # Moving numbers to the end of the string/file_name
    parent_dir = os.path.dirname(data).split('\\')[-1]
    file_name = parent_dir+'_x_'+file_name
        
#     # Parent dir
#     dir1_name = os.path.basename(os.path.dirname(data))
#     # grand parent dir
#     dir2_name = os.path.basename(os.path.dirname(os.path.dirname(data)))

    # Replce all invalid characters in the name
    file_name = file_name.replace("-", "_")
    file_name = file_name.replace(" ", "_")
    file_name = file_name.replace("(", "")
    file_name = file_name.replace(")", "")
    file_name = file_name.replace(".", "")

    xl_file = pd.ExcelFile(data,engine='openpyxl')
    
    for sheet in xl_file.sheet_names:
        if len(xl_file.sheet_names)>1:
            sheet_str = sheet
            sheet_str = sheet_str.replace("-", "_")
            sheet_str = sheet_str.replace(" ", "_")
            sheet_str = sheet_str.replace("(", "")
            sheet_str = sheet_str.replace(")", "")
            sheet_str = sheet_str.replace(".", "")

            df_name = file_name+'_x_'+sheet_str
            
        else:
            df_name = file_name

        # Test: Check if the same date is already present in the current dict key
        if df_name in df_yield:
            print(f'A file with the same name {df_name} has already been imported. \n Please check if there is duplication of data.')
            raise NameError
    
        df_yield.append(df_name)
        print(df_name)
        # Adding new and old names+path to a dict for record
        yield_import_to_from[df_name] = data
        
        locals()[df_name] = pd.read_excel(data, sheet_name=sheet, engine='openpyxl')


2019_Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Data


C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


2019_Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Field_map
2019_Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_lsmeans
2019_Vollebekk_Graminor_x_19TvPhenores_x_Staur_kart
2019_Vollebekk_Graminor_x_19TvPhenores_x_Staur_res
2019_Vollebekk_Graminor_x_19TvPhenores_x_Vollebekk_kart
2019_Vollebekk_Graminor_x_19TvPhenores_x_Vollebekk_res
2019_Vollebekk_Graminor_x_Graminor_Staur_Vollebekk_2019_x_Staur_kart_new
2019_Vollebekk_Graminor_x_Graminor_Staur_Vollebekk_2019_x_Staur_res_new
2019_Vollebekk_Graminor_x_Graminor_Staur_Vollebekk_2019_x_Vollebekk_kart_new
2019_Vollebekk_Graminor_x_Graminor_Staur_Vollebekk_2019_x_Vollebekk_res_new
2019_Vollebekk_Masbasis_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_data


C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


2019_Vollebekk_Masbasis_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_lsmeans
2019_Vollebekk_Masbasis_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_Sheet1
2019_Vollebekk_Masbasis_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_Sheet2
2019_Vollebekk_Masbasis_x_Masbasis_vollebekk_trial_data_2020_x_data
2019_Vollebekk_Masbasis_x_Masbasis_vollebekk_trial_data_2020_x_lsmeans
2019_Vollebekk_Masbasis_x_Masbasis_vollebekk_trial_data_2020_x_lsmeans_lmer
2020_Staur_Graminor_x_Staur_Graminor_Masbasis_2020_x_Map
2020_Staur_Graminor_x_Staur_Graminor_Masbasis_2020_x_Res
2020_Staur_Graminor_x_Staur_Graminor_Masbasis_2020_x_Sheet1
2020_Staur_Masbasis_x_20BMLFN3___MASBASIS_avlingsforsøk_Staur_2020_lodging_data_x_Feltbok
2020_Staur_Masbasis_x_20BMLFN3___MASBASIS_avlingsforsøk_Staur_2020_lodging_data_x_kart
2020_Staur_Masbasis_x_Masbasis_2020_staur_x_Feltbok
2020_Staur_Masbasis_x_Masbasis_2020_staur_x_kart
2020_Staur_Masbasis_x_Staur_maturity_heading_yield_2020_x_Feltbok
2020_Staur_Masbasis_

## Identifying the sheets with yield data

In [39]:
# # Listing datasets for reference
# for df in all_df_std:
#     print(df, locals()[df].shape, 'Min Plot_ID value:', locals()[df]['Plot_ID'].min(), 'Max Plot_ID value:', locals()[df]['Plot_ID'].max())

Inspecting the datasets from Graminor, it can be noted that the plot id of all the plots of Graminor correspond to the range from 100 to 900, which are aparently all in Gramminor. Staur Plot IDs are 4 digit. So, we do not any bands data from Staur at this point.

In [39]:
# Ask: Yield data not available

### Graminor 2019 
#### Vollebekk
19TvPhenores

In [40]:
df_yield

['2019_Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Data',
 '2019_Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Field_map',
 '2019_Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_lsmeans',
 '2019_Vollebekk_Graminor_x_19TvPhenores_x_Staur_kart',
 '2019_Vollebekk_Graminor_x_19TvPhenores_x_Staur_res',
 '2019_Vollebekk_Graminor_x_19TvPhenores_x_Vollebekk_kart',
 '2019_Vollebekk_Graminor_x_19TvPhenores_x_Vollebekk_res',
 '2019_Vollebekk_Graminor_x_Graminor_Staur_Vollebekk_2019_x_Staur_kart_new',
 '2019_Vollebekk_Graminor_x_Graminor_Staur_Vollebekk_2019_x_Staur_res_new',
 '2019_Vollebekk_Graminor_x_Graminor_Staur_Vollebekk_2019_x_Vollebekk_kart_new',
 '2019_Vollebekk_Graminor_x_Graminor_Staur_Vollebekk_2019_x_Vollebekk_res_new',
 '2019_Vollebekk_Masbasis_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_data',
 '2019_Vollebekk_Masbasis_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_lsmeans',
 '2019_Vollebekk_Masbasis_x_19BMLGI1___MASBASIS_y

In [40]:
# # Checking if the sheets from the file '19TvPhenores' and '19TvPhenores_pedigree' are the same

# # Graminor_2019_x_19TvPhenores vs Staur_2019_x_19TvPhenores
# # Staur Check
# print(Graminor_2019_x_19TvPhenores_x_Staur_res.equals(Staur_2019_x_19TvPhenores_pedigree_x_Staur_res))
# # Reverse check
# print(Staur_2019_x_19TvPhenores_pedigree_x_Staur_res.equals(Graminor_2019_x_19TvPhenores_x_Staur_res))
# # Vollebekk check
# print(Graminor_2019_x_19TvPhenores_x_Vollebekk_res.equals(Staur_2019_x_19TvPhenores_pedigree_x_Vollebekk_res))
# # Reverse check
# print(Staur_2019_x_19TvPhenores_pedigree_x_Vollebekk_res.equals(Graminor_2019_x_19TvPhenores_x_Vollebekk_res))

# # Graminor_2019_x_19TvPhenores vs Staur_2019_x_19TvPhenores_pedigree
# # Staur Check
# print(Graminor_2019_x_19TvPhenores_x_Staur_res.equals(Staur_2019_x_19TvPhenores_pedigree_x_Staur_res))
# # Reverse check
# print(Staur_2019_x_19TvPhenores_pedigree_x_Staur_res.equals(Graminor_2019_x_19TvPhenores_x_Staur_res))
# # Vollebekk check
# print(Graminor_2019_x_19TvPhenores_x_Vollebekk_res.equals(Staur_2019_x_19TvPhenores_pedigree_x_Vollebekk_res))
# # Reverse check
# print(Staur_2019_x_19TvPhenores_pedigree_x_Vollebekk_res.equals(Graminor_2019_x_19TvPhenores_x_Vollebekk_res))

# # Staur_2019_x_19TvPhenores vs Staur_2019_x_19TvPhenores_pedigree
# # Staur Check
# print(Staur_2019_x_19TvPhenores_x_Vollebekk_res.equals(Staur_2019_x_19TvPhenores_pedigree_x_Vollebekk_res))
# # Reverse check
# print(Staur_2019_x_19TvPhenores_pedigree_x_Vollebekk_res.equals(Staur_2019_x_19TvPhenores_x_Vollebekk_res))
# # Vollebekk check
# print(Staur_2019_x_19TvPhenores_x_Staur_res.equals(Staur_2019_x_19TvPhenores_pedigree_x_Staur_res))
# # Reverse check
# print(Staur_2019_x_19TvPhenores_pedigree_x_Staur_res.equals(Staur_2019_x_19TvPhenores_x_Staur_res))

Turns out that the file '19TvPhenores' in Graminor_2019 and in Staur_2019, and 19TvPhenores_pedigree file in Staur_2019 have the same sheets regarding grain yield data. So we will use the '19TvPhenores' in Graminor_2019 directory.

19TvPhenores_pedigree and 19TvPhenores files in Staur 2019 have been removed.


##### Graminor_2019_x_Graminor_Staur_Vollebekk_2019 vs Graminor_2019_x_19TvPhenores  


In [41]:
# # Graminor_2019_x_Graminor_Staur_Vollebekk_2019 vs Graminor_2019_x_19TvPhenores

# Staur res Check
print(Graminor_2019_x_19TvPhenores_x_Staur_res.equals(Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Staur_res_new))
# Reverse check
print(Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Staur_res_new.equals(Graminor_2019_x_19TvPhenores_x_Staur_res))
# Vollebekk res check
print(Graminor_2019_x_19TvPhenores_x_Vollebekk_res.equals(Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Vollebekk_res_new))
# Reverse check
print(Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Vollebekk_res_new.equals(Graminor_2019_x_19TvPhenores_x_Vollebekk_res))


False
False
True
True


KART are same in both files. Graminor res has been corrected and found to be same.  
Graminor_2019_x_Graminor_Staur_Vollebekk_2019 has more rows in res sheet for staur

ToDo: Staur: The data for Staur is totally different in both files. Even the plant varieties planted in each subplot are also different. Check

In [70]:
# Removing the files that have been inspected

inspected_df_graminor19 = ['Graminor_2019_x_19TvPhenores_x_Staur_kart',
 'Graminor_2019_x_19TvPhenores_x_Vollebekk_kart',
 'Graminor_2019_x_19TvPhenores_x_Vollebekk_res',
 'Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Staur_kart_new',
 'Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Vollebekk_kart_new',
 'Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Vollebekk_res_new']

df_yield = [x for x in df_yield if x not in inspected_df_graminor19]
df_yield

#### Staur

### Graminor 2020

#### Vollebekk

In [54]:
df_yield

['Graminor_2019_x_19TvPhenores_x_Staur_res',
 'Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Staur_res_new',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_data',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_lsmeans',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_Sheet1',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_Sheet2',
 'Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Data',
 'Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Field_map',
 'Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_lsmeans',
 'Masbasis_x_20BMLGI1_2020_tm_x_data',
 'Masbasis_x_20BMLGI1_2020_tm_x_lsmeans',
 'Masbasis_x_20BMLGI1_2020_tm_x_lsmeans_lmer',
 'Masbasis_x_Masbasis_Mica_2020_all_dates_MEDIAN_DP_x_Sheet1',
 'Masbasis_x_Masbasis_Mica_2020_all_dates_MEDIAN_DP_x_biomass',
 'Masbasis_x_Masbasis_Mica_2020_all_dates_MEDIAN_DP_x_Without_loadging',
 'Robot_x_ROBOT_2020_x_raw',
 'Robot_

In [44]:
Graminor_x_2020TGraminor_Vollebekk_res

,20T1A,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,%,Avl.,Hl.-,%.1
0,r_expt,r_location,loccode,plot,bloc,ibloc,entry,NaN,name,pedigree,Vann,kg/daa,vekt,Prot
1,20T1A_4,Vollebekk,04,101,1,1,1,NaN,Zebra,Ralle/Dragon,15.9281,654.708,78.8,11.5
2,20T1A_4,Vollebekk,04,102,1,1,19,NaN,GN18666,GN04528/GN03509,15.7755,709.595,78.3,10.7
3,20T1A_4,Vollebekk,04,103,1,1,11,NaN,GN15590,Demonstrant/SW51114,16.2226,707.101,80.1,10.6
4,20T1A_4,Vollebekk,04,104,1,1,5,NaN,Mirakel,SW38337/NK98533//NK98535,15.3251,724.959,78.1,10.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,20T16B4,Vollebekk,04,896,2,10,23,NaN,GN20696,Saar/2*Avle,17.0125,663.9,NaN,NaN
797,20T16B4,Vollebekk,04,897,2,10,20,NaN,GN20693,ONPMSYDER 5/GN08588,17.8175,722.332,NaN,NaN
798,20T16B4,Vollebekk,04,898,2,10,6,NaN,GN20679,SW71139/GN06600,18.1271,782.997,NaN,NaN
799,20T16B4,Vollebekk,04,899,2,10,9,NaN,GN20682,GN07574/SW71139,17.6732,712.353,NaN,NaN


In [71]:
# Removing the files that have been inspected

inspected_df_graminor20 = ['Graminor_x_2020TGraminor_Vollebekk_res']

df_yield = [x for x in df_yield if x not in inspected_df_graminor20]
df_yield

['Graminor_2019_x_19TvPhenores_x_Staur_res',
 'Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Staur_res_new',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_data',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_lsmeans',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_Sheet1',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_Sheet2',
 'Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Data',
 'Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Field_map',
 'Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_lsmeans',
 'Masbasis_x_20BMLGI1_2020_tm_x_data',
 'Masbasis_x_20BMLGI1_2020_tm_x_lsmeans',
 'Masbasis_x_20BMLGI1_2020_tm_x_lsmeans_lmer',
 'Masbasis_x_Masbasis_Mica_2020_all_dates_MEDIAN_DP_x_Sheet1',
 'Masbasis_x_Masbasis_Mica_2020_all_dates_MEDIAN_DP_x_biomass',
 'Masbasis_x_Masbasis_Mica_2020_all_dates_MEDIAN_DP_x_Without_loadging',
 'Robot_x_ROBOT_2020_x_raw',
 'Robot_

Yield data is not available

#### Staur

### Masbasis 2019

#### Vollebekk

In [55]:
print(Masbasis_2019_x_Field_data_2019['GrainYield'].equals(Masbasis_2019_x_NEW_Field_data_2019['GrainYield']))
# Reverse check
print(Masbasis_2019_x_NEW_Field_data_2019['GrainYield'].equals(Masbasis_2019_x_Field_data_2019['GrainYield']))

True
True


Masbasis_2019_x_Field_data_2019 and Masbasis_2019_x_NEW_Field_data_2019 have same grain yield data.

For Masbasis_2019_x_Masbasis_2019_ForDP_median, the difference is extremely small. So we will use Masbasis_2019_x_Field_data_2019.

In [56]:
Masbasis_2019_x_Masbasis_2019_ForDP_median['GrainYield']==(Masbasis_2019_x_NEW_Field_data_2019['GrainYield'])
pd.set_option("display.precision", 14)
comparision = Masbasis_2019_x_Masbasis_2019_ForDP_median['GrainYield'].compare(Masbasis_2019_x_NEW_Field_data_2019['GrainYield'])
# comparision.iloc[:,0:1].sub(comparision.iloc[:,1:2])
comparision.head()

,self,other
10,486.66666666666657,486.66666666666663
14,414.66666666666657,414.66666666666663
15,480.00000000000011,480.00000000000006
25,497.33333333333343,497.33333333333337
31,457.33333333333343,457.33333333333337


In [72]:
df_yield

['Graminor_2019_x_19TvPhenores_x_Staur_res',
 'Graminor_2019_x_Graminor_Staur_Vollebekk_2019_x_Staur_res_new',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_data',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_lsmeans',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_Sheet1',
 'Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_Sheet2',
 'Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Data',
 'Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_Field_map',
 'Staur_Masbasis_x_19BMLFN3___MASBASIS_yield_trial_Staur_2019_x_lsmeans',
 'Masbasis_x_20BMLGI1_2020_tm_x_data',
 'Masbasis_x_20BMLGI1_2020_tm_x_lsmeans',
 'Masbasis_x_20BMLGI1_2020_tm_x_lsmeans_lmer',
 'Masbasis_x_Masbasis_Mica_2020_all_dates_MEDIAN_DP_x_Sheet1',
 'Masbasis_x_Masbasis_Mica_2020_all_dates_MEDIAN_DP_x_biomass',
 'Masbasis_x_Masbasis_Mica_2020_all_dates_MEDIAN_DP_x_Without_loadging',
 'Robot_x_ROBOT_2020_x_raw',
 'Robot_

In [74]:
Masbasis_2019_x_19BMLGI1___MASBASIS_yield_trial_Vollebekk_2019_x_lsmeans

,Line,MASBASIS 2015,Name,DH,StdErr,DM,StdErr.1,PH,StdErr.2,GrainYield,StdErr.3,Protein,StdErr.4,TW,StdErr.5,TKW,StdErr.6
0,5,1003,Bastian,65.57490000000000,0.3821,107.41000000000000,0.4893,91.45200000000000,1.9445,558.12000000000000,15.5952,10.9795,0.2265,79.61369999999999,0.3552,34.3422,0.8559
1,3,1005,Bjarne,67.44350000000000,0.3850,109.19000000000000,0.4848,81.92659999999999,1.9396,614.54999999999995,15.4297,10.6130,0.2274,79.32129999999999,0.3563,36.0514,0.8509
2,1,1006,Tjalve,68.51360000000000,0.3821,108.19000000000000,0.4892,95.49379999999999,1.9445,551.54999999999995,15.5794,11.1814,0.2264,77.44390000000000,0.3551,35.6983,0.8559
3,2,1009,Avle,67.68290000000000,0.3811,107.80000000000000,0.4895,91.29640000000001,1.9446,570.34000000000003,15.5899,10.7511,0.2261,77.81290000000000,0.3545,33.2953,0.8560
4,4,1011,Zebra,66.88580000000000,0.3828,109.11000000000000,0.4888,109.20999999999999,1.9445,635.24000000000001,15.5656,10.5530,0.2274,79.73540000000000,0.3564,41.8470,0.8557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,1632,1632,TRAP#1/BOW//TAIGU DERIVATIVE,69.58580000000001,0.3811,109.04000000000001,0.4759,88.23210000000000,1.9328,411.54000000000002,15.1224,12.5090,0.2252,78.46030000000000,0.3525,38.5857,0.8425
297,1633,1633,IVAN/6/SABUF/5/BCN/4/RABI//GS/CRA/3/AE.SQUARRO...,74.25690000000000,0.5300,110.83000000000000,0.6617,88.23410000000000,2.6665,439.41000000000003,21.1052,11.5865,0.2778,80.82259999999999,0.4884,39.8087,1.1717
298,1634,1634,GAMENYA,66.14480000000000,0.5304,108.02000000000000,0.6624,101.18000000000001,2.6666,432.63000000000000,21.7812,10.6785,0.2782,73.93400000000000,0.4889,38.2115,1.1720
299,1635,1635,WHEAR/2*KRONSTAD F2004,69.72929999999999,0.5285,112.31000000000000,0.6627,99.68370000000000,2.6666,524.11000000000001,21.1400,10.4479,0.2773,79.87050000000001,0.4877,37.1370,1.1722


In [ ]:
# Removing the files that have been inspected

inspected_df_mas19 = ['Graminor_x_2020TGraminor_Vollebekk_res']

df_yield = [x for x in df_yield if x not in inspected_df_graminor20]
df_yield

#### Staur

### Masbasis 2020

#### Vollebekk

In [57]:
Masbasis_x_20BMLGI1_2020_tm_x_data['GrainYield'].dropna()

4      713.33333333333337
5      677.33333333333337
6      361.33333333333331
7      697.33333333333337
8      664.00000000000000
              ...        
679    646.66666666666663
680    478.66666666666669
681    732.00000000000000
682    600.00000000000000
683    714.66666666666663
Name: GrainYield, Length: 571, dtype: float64

In [58]:
Masbasis_x_20BMLGI1_2020_tm_x_lsmeans_lmer['GrainYield']

0      692.64982695641095
1      677.86196971022900
2      662.28221171870598
3      667.96322901550400
4      654.33320147100403
              ...        
283    517.81248520565703
284    593.26000855945904
285    499.96906032742100
286    539.15045569047300
287    535.09866305721403
Name: GrainYield, Length: 288, dtype: float64

Analysing the grain yields in thee above sheets for Masbasis 2020, 'Masbasis_x_20BMLGI1_2020_tm_x_data' seems to have more data than the other one. Also, the second sheet seems to have the yields for different varieties, not for individual subplots. If a varaiety is sown in two subplots, the secondd file only has one entry for that variety. So, it is decided to use 'Masbasis_x_20BMLGI1_2020_tm_x_data' tto get the grain yield datata.

#### Staur

### Robot 2020

In [59]:
Robot_x_ROBOT_2020_x_lsmeans_EntryRobot_x_Fert.shape

(48, 15)

In [60]:
Robot_x_ROBOT_2020_x_raw.shape

(96, 29)

'Robot_x_ROBOT_2020_x_raw' has more data points, and all of them correspond to the number of subplots on the Robot field of 2020. So this is the one used for grain yield data.

Following datasets have grain yield data  

Staur 2019: Graminor_2019_x_19TvPhenores_x_Staur_res  
Vollebekk 2019: Graminor_2019_x_19TvPhenores_x_Vollebekk_res  
Masbasis 2020: Masbasis_x_20BMLGI1_2020_tm_x_data  
Robot 2020: Robot_x_ROBOT_2020_x_raw  
Masbasis 2019: Masbasis_2019_x_Field_data_2019  

In [ ]:
# Yield data is required for the following fields/years
sorted_field_year_dict.keys()

### Masbasis 2021

#### Vollebekk

#### Staur

## ToDo: Automate: Exporting the path of datasets with yield data

In [ ]:
yield_df = {
    'Staur 2019': 'Graminor_2019_x_19TvPhenores_x_Staur_res',
    'Graminor 2019': 'Graminor_2019_x_19TvPhenores_x_Vollebekk_res',
    'Masbasis 2020': 'Masbasis_x_20BMLGI1_2020_tm_x_data',
    'Robot 2020': 'Robot_x_ROBOT_2020_x_raw',
    'Masbasis 2019': 'Masbasis_2019_x_Field_data_2019'
}
# yield_import_to_from
yield_df_path = {}
for field, df in yield_df.items():
    shrinked_path = yield_import_to_from[df].split(username)[1]
    yield_df_path[field] = [df, shrinked_path]
pprint(yield_df_path)

In [ ]:
# Make sure the folder/dir is there. If not, create one
os.makedirs(path, exist_ok=True)
import json
a_file = open(path+'yield_df.json', "w")
json.dump(yield_df_path, a_file)
a_file.close()

# a_file = open("Data\std_columns.json", "r")
# output = a_file.read()
# a_file.close()
# print(output)

## Adding yield data to all datasets

In [61]:
# Defining expty list to collect names of the final df with yields
df_w_yields = []

### Graminor 2019

In [66]:
# Graminor_2019_x_19TvPhenores_x_Vollebekk_res.head()

In [63]:
# Dropping first row/ unnamed column headings and replacing them with the second row.
Graminor_2019_x_19TvPhenores_x_Vollebekk_res_temp = Graminor_2019_x_19TvPhenores_x_Vollebekk_res.copy()
Graminor_2019_x_19TvPhenores_x_Vollebekk_res_temp.columns = Graminor_2019_x_19TvPhenores_x_Vollebekk_res.iloc[0]
Graminor_2019_x_19TvPhenores_x_Vollebekk_res_new = Graminor_2019_x_19TvPhenores_x_Vollebekk_res_temp[1:]
# Graminor_2019_x_19TvPhenores_x_Vollebekk_res_new.head()

In [64]:
Graminor_2019_x_19TvPhenores_x_Vollebekk_res_new

,r_expt,r_location,plot,bloc,ibloc,entry,NaN,name,pedigree,Legde,høyde,Juli,Aug.,kg/daa
1,19T1A 4,Vollebekk,101,1,1,1,NaN,Zebra,Ralle/Dragon,NaN,107.5,0,9,499.6244402277
2,19T1A 4,Vollebekk,102,1,1,11,NaN,GN14547,SW45126/NK01533,NaN,88.5,0,8,515.53275142315
3,19T1A 4,Vollebekk,103,1,1,13,NaN,Tarrafal,NaN,NaN,87,0,10,529.50102466793
4,19T1A 4,Vollebekk,104,1,1,14,NaN,GN12760,NK02529/NK01533//NK01568,NaN,95,0,7,544.50398481973
5,19T1A 4,Vollebekk,105,1,1,4,NaN,Bjarne,SvB87293/Bastian,NaN,79.5,-1,8,529.50102466793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,19T15B,Vollebekk,871,4,20,10,NaN,GN19588,QUARNA/BAJASS-5//Berserk 4,NaN,78.5,1,10,542.82079696395
597,19T15B,Vollebekk,872,4,20,24,NaN,GN19602,QUARNA/NK01513//Berserk,NaN,88,5,11,469.68227703985
598,19T15B,Vollebekk,873,4,20,9,NaN,GN19587,QUARNA/BAJASS-5//Berserk 4,NaN,79,0,10,594.68265654649
599,19T15B,Vollebekk,874,4,20,21,NaN,GN19599,Bombona/NK01513//Berserk,NaN,90.5,4,11,528.06011385199


In [ ]:
field = 'Graminor_2019'
field_plot_id = 'Plot_ID'
ref_df_yield = Graminor_2019_x_19TvPhenores_x_Vollebekk_res_new
red_df_id = 'plot'


cols2add_dict = {'GrainYield':'kg/daa',
                 'Name':'name',
#                  'CodeName':'CodeName',
                 'Pedigree':'pedigree',
#                  'Line':'Line',
#                  'Heading_Date':'Hd_date_jd',
#                  'Maturity_Date':'maturity_dates',
#                  'Days2Heading':'DH',
#                  'Days2Maturity':'DM',
#                  'Lodging':'Lodging_images'
                }


for date in field_year_dict[field]:
    temp_df = locals()[field[:-4]+date].copy()
    temp_ref_df = ref_df_yield.copy()
    
    temp_df.set_index(field_plot_id, inplace=True)
    temp_ref_df.set_index(red_df_id, inplace=True)
    for field_df_col, ref_df_col in cols2add_dict.items():
        temp_df[field_df_col]=temp_ref_df[ref_df_col]
    new_df_name = field[:-4]+date+'_yield'
    locals()[new_df_name] = temp_df.reset_index()
    df_w_yields.append(new_df_name)
    print(new_df_name)
    display(locals()[new_df_name].head())

### Graminor_2020

In [68]:
Graminor_x_2020TGraminor_Vollebekk_res.head()

,20T1A,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,%,Avl.,Hl.-,%.1
0,r_expt,r_location,loccode,plot,bloc,ibloc,entry,NaN,name,pedigree,Vann,kg/daa,vekt,Prot
1,20T1A_4,Vollebekk,04,101,1,1,1,NaN,Zebra,Ralle/Dragon,15.928070175439,654.70815939279,78.8,11.5
2,20T1A_4,Vollebekk,04,102,1,1,19,NaN,GN18666,GN04528/GN03509,15.775495495495,709.5954459203,78.3,10.7
3,20T1A_4,Vollebekk,04,103,1,1,11,NaN,GN15590,Demonstrant/SW51114,16.222571942446,707.10056925996,80.1,10.6
4,20T1A_4,Vollebekk,04,104,1,1,5,NaN,Mirakel,SW38337/NK98533//NK98535,15.325088652482,724.95863377609,78.1,10.8


In [67]:
# Dropping first row/ unnamed column headings and replacing them with the second row.
Graminor_x_2020TGraminor_Vollebekk_res_temp = Graminor_x_2020TGraminor_Vollebekk_res.copy()
Graminor_x_2020TGraminor_Vollebekk_res_temp.columns = Graminor_x_2020TGraminor_Vollebekk_res.iloc[0]
Graminor_x_2020TGraminor_Vollebekk_res_new = Graminor_x_2020TGraminor_Vollebekk_res_temp[1:]
Graminor_x_2020TGraminor_Vollebekk_res_new.head()

,r_expt,r_location,loccode,plot,bloc,ibloc,entry,NaN,name,pedigree,Vann,kg/daa,vekt,Prot
1,20T1A_4,Vollebekk,04,101,1,1,1,NaN,Zebra,Ralle/Dragon,15.928070175439,654.70815939279,78.8,11.5
2,20T1A_4,Vollebekk,04,102,1,1,19,NaN,GN18666,GN04528/GN03509,15.775495495495,709.5954459203,78.3,10.7
3,20T1A_4,Vollebekk,04,103,1,1,11,NaN,GN15590,Demonstrant/SW51114,16.222571942446,707.10056925996,80.1,10.6
4,20T1A_4,Vollebekk,04,104,1,1,5,NaN,Mirakel,SW38337/NK98533//NK98535,15.325088652482,724.95863377609,78.1,10.8
5,20T1A_4,Vollebekk,04,105,1,1,24,NaN,GN18751,QUARNA/GN03531,16.845059625213,740.97836812144,77.7,10.6


In [69]:
field = 'Graminor_2020'
field_plot_id = 'Plot_ID'
ref_df_yield = Graminor_x_2020TGraminor_Vollebekk_res_new
red_df_id = 'plot'

cols2add_dict = {'GrainYield':'kg/daa',
                 'Name':'name',
#                  'CodeName':'CodeName',
                 'Pedigree':'pedigree',
#                  'Line':'Line',
#                  'Heading_Date':'Hd_date_jd',
#                  'Maturity_Date':'maturity_dates',
#                  'Days2Heading':'DH',
#                  'Days2Maturity':'DM',
#                  'Lodging':'Lodging_images'
                }


for date in field_year_dict[field]:
    temp_df = locals()[field[:-4]+date].copy()
    temp_ref_df = ref_df_yield.copy()
    
    temp_df.set_index(field_plot_id, inplace=True)
    temp_ref_df.set_index(red_df_id, inplace=True)
    for field_df_col, ref_df_col in cols2add_dict.items():
        temp_df[field_df_col]=temp_ref_df[ref_df_col]
    new_df_name = field[:-4]+date+'_yield'
    locals()[new_df_name] = temp_df.reset_index()
    df_w_yields.append(new_df_name)
    print(new_df_name)
    display(locals()[new_df_name].head())

Graminor_040720_yield


,Plot_ID,Blue_Mean_Value,Blue_Median_Value,Blue_Std_Dev_Value,Green_Mean_Value,Green_Median_Value,Green_Std_Dev_Value,Red_Mean_Value,Red_Median_Value,Red_Std_Dev_Value,RedEdge_Mean_Value,RedEdge_Median_Value,RedEdge_Std_Dev_Value,NIR_Mean_Value,NIR_Median_Value,NIR_Std_Dev_Value,GrainYield,Name,Pedigree
0,101,0.01785124426339,0.01731912605465,0.00503656913861,0.04316562577981,0.04214972630143,0.01056329130895,0.02178064962805,0.02136499434710,0.00515298584236,0.11197198990172,0.11029557138681,0.02075150750696,0.48582822174294,0.48360610008240,0.06080637707182,654.70815939279,Zebra,Ralle/Dragon
1,102,0.01732559858581,0.01709566265345,0.00412828927984,0.04489375691789,0.04406672716141,0.00964595604654,0.02146992840469,0.02111094258726,0.00476094477466,0.11374823961286,0.11248160898685,0.01752475032538,0.49904403830504,0.50077459216118,0.04689259690942,709.5954459203,GN18666,GN04528/GN03509
2,103,0.01779076703092,0.01724500674754,0.00494665146184,0.04828509524885,0.04677460156381,0.01231924298816,0.02239840297244,0.02177555020899,0.00543271682182,0.12127026558460,0.11889328435063,0.02300496857624,0.50042118751948,0.50010347366333,0.06193161416382,707.10056925996,GN15590,Demonstrant/SW51114
3,104,0.01614745066061,0.01585669536144,0.00351963312038,0.04309530835470,0.04220090247691,0.00864687516408,0.01938866181705,0.01899677887559,0.00369131814613,0.11692252363026,0.11562358587980,0.01790956473573,0.50405047827974,0.50624313950539,0.04929491721741,724.95863377609,Mirakel,SW38337/NK98533//NK98535
4,105,0.01904844114652,0.01848565228283,0.00519994870793,0.05067893788090,0.04919448308647,0.01231680829260,0.02236159910939,0.02177054807544,0.00544669129795,0.12990068230758,0.12781217694283,0.02330782278256,0.56038144935303,0.56282949447632,0.05873137707639,740.97836812144,GN18751,QUARNA/GN03531


Graminor_040820_yield


,Plot_ID,Blue_Mean_Value,Blue_Median_Value,Blue_Std_Dev_Value,Green_Mean_Value,Green_Median_Value,Green_Std_Dev_Value,Red_Mean_Value,Red_Median_Value,Red_Std_Dev_Value,RedEdge_Mean_Value,RedEdge_Median_Value,RedEdge_Std_Dev_Value,NIR_Mean_Value,NIR_Median_Value,NIR_Std_Dev_Value,GrainYield,Name,Pedigree
0,101,0.03080202141778,0.02888692356646,0.01126231863643,0.06331523748071,0.06124439090490,0.01996896454359,0.09230243986102,0.08660537749529,0.03105507878195,0.17395000717765,0.16973912715912,0.04181885611259,0.31472062188135,0.30749645829201,0.06138482561309,654.70815939279,Zebra,Ralle/Dragon
1,102,0.02994548378188,0.02778669912368,0.01171717891633,0.06045146374125,0.05822783522308,0.01934226326273,0.09098579468548,0.08270449191332,0.03274571331715,0.16541110998700,0.16041847318411,0.03996604335603,0.29753544097807,0.28869172930717,0.05756787840363,709.5954459203,GN18666,GN04528/GN03509
2,103,0.03150665360917,0.02882610075176,0.01298940114176,0.06072599327545,0.05806675925851,0.02083064394317,0.08992066418909,0.08406268805265,0.03441703102692,0.16005584374937,0.15537573397160,0.04114175859206,0.28656492635971,0.27746567130089,0.06058272001647,707.10056925996,GN15590,Demonstrant/SW51114
3,104,0.03962737313458,0.03581303358078,0.01961923450805,0.07839145754922,0.07255017384887,0.03055626909658,0.10604226548866,0.09702980145812,0.04371585061603,0.19669524414641,0.18787904083729,0.05728325857278,0.32495923356092,0.31686189770698,0.07062041992232,724.95863377609,Mirakel,SW38337/NK98533//NK98535
4,105,0.03429953950622,0.03172615915537,0.01428203601431,0.07499742881638,0.07085644453764,0.02673534011350,0.09279282041724,0.08739325404167,0.03207677884582,0.19735332438324,0.18942515552044,0.05411010649480,0.35199003485884,0.34768979251385,0.07619284034753,740.97836812144,GN18751,QUARNA/GN03531


Graminor_070720_yield


...

,Plot_ID,Blue_Mean_Value,Blue_Median_Value,Blue_Std_Dev_Value,Green_Mean_Value,Green_Median_Value,Green_Std_Dev_Value,Red_Mean_Value,Red_Median_Value,Red_Std_Dev_Value,RedEdge_Mean_Value,RedEdge_Median_Value,RedEdge_Std_Dev_Value,NIR_Mean_Value,NIR_Median_Value,NIR_Std_Dev_Value,GrainYield,Name,Pedigree
0,151,0.03213385047758,0.02500713057816,0.02109973232802,0.06484386590164,0.06103284284473,0.02560158724755,0.04464433494469,0.02360297739506,0.04140486919470,0.15080864585243,0.14782261848450,0.03201022599812,0.47119734252987,0.49470546841621,0.13172726748099,708.4544971537,GN20540,SW71139/GN07501
1,152,0.03335237292255,0.02698950842023,0.01772455196957,0.06672974564584,0.06481597572565,0.02094243555161,0.04474580737820,0.02611383050680,0.03631227567283,0.15255196559753,0.14868681132793,0.02983908579898,0.46646194007602,0.49543772637844,0.13988956190871,666.22785578748,GN20536,SW51069/GN03509
2,153,0.03043436059387,0.02630499657243,0.01363782760163,0.06200760242408,0.06199061125517,0.01698878814119,0.04061590337346,0.02458008751273,0.02926316298477,0.14560045677997,0.13743014633656,0.03132187632948,0.46057262414494,0.48579984903336,0.14678913578655,654.71149905123,GN20534,GN04526/GN08581
3,154,0.03444844642793,0.02678878419101,0.02109226058293,0.06559076689600,0.06211842969060,0.02585696018635,0.04753483436194,0.02538747154176,0.04181523538322,0.14841092445349,0.14423263072967,0.03437751773452,0.46220747147682,0.46459522843361,0.14432932843426,621.48614800759,GN20529,Krabat/GN08536
4,155,0.03228616375828,0.02933445945382,0.01248274273806,0.06685170656152,0.06688608974218,0.01640636256656,0.04128509952550,0.02741109300405,0.02796708472376,0.15754510767176,0.15244185924530,0.03242753363076,0.48712174423463,0.53210029006004,0.15142815106172,674.96440227704,Demonstrant,T1005/NK93512


Graminor_200720_yield


,Plot_ID,Blue_Mean_Value,Blue_Median_Value,Blue_Std_Dev_Value,Green_Mean_Value,Green_Median_Value,Green_Std_Dev_Value,Red_Mean_Value,Red_Median_Value,Red_Std_Dev_Value,RedEdge_Mean_Value,RedEdge_Median_Value,RedEdge_Std_Dev_Value,NIR_Mean_Value,NIR_Median_Value,NIR_Std_Dev_Value,GrainYield,Name,Pedigree
0,151,0.02159986026629,0.02052104379982,0.00601684148737,0.06343362275714,0.06073131039739,0.01733408407285,0.03327499431132,0.03193806298077,0.00912226998864,0.14601712852760,0.14240511506796,0.02910731964140,0.38715196228318,0.38521130383015,0.04442337585817,708.4544971537,GN20540,SW71139/GN07501
1,152,0.02468500379982,0.02352127619088,0.00724769961060,0.06926058144930,0.06676716357470,0.01767773452256,0.04497625586361,0.04440850950778,0.00995424301235,0.15576125339014,0.15232443064451,0.02910502649312,0.38252880708523,0.37749837338924,0.05366883658171,666.22785578748,GN20536,SW51069/GN03509
2,153,0.02194380651548,0.02119263634086,0.00577078871897,0.06260020861200,0.06027925387025,0.01635446038164,0.03560792489691,0.03469301760197,0.00833960446334,0.14511067557380,0.14231227338314,0.02769542010507,0.37354604076689,0.37120103836060,0.04496683610180,654.71149905123,GN20534,GN04526/GN08581
3,154,0.02186740285061,0.02113559655845,0.00576407994523,0.06422586687028,0.06225795857608,0.01534806261332,0.03770472429041,0.03737545385957,0.00946308889664,0.14671362074999,0.14357797801495,0.02554821718027,0.37362543327913,0.37263041734695,0.04996456896537,621.48614800759,GN20529,Krabat/GN08536
4,155,0.02082497385994,0.01975152548402,0.00570507796918,0.06007707453441,0.05702294781804,0.01596157684472,0.03397111573818,0.03292134776712,0.00831867795230,0.13469903893354,0.13056176155806,0.02574132912536,0.34099238840548,0.33940501511097,0.04313883804052,674.96440227704,Demonstrant,T1005/NK93512


Graminor_240620_yield


,Plot_ID,Blue_Mean_Value,Blue_Median_Value,Blue_Std_Dev_Value,Green_Mean_Value,Green_Median_Value,Green_Std_Dev_Value,Red_Mean_Value,Red_Median_Value,Red_Std_Dev_Value,RedEdge_Mean_Value,RedEdge_Median_Value,RedEdge_Std_Dev_Value,NIR_Mean_Value,NIR_Median_Value,NIR_Std_Dev_Value,GrainYield,Name,Pedigree
0,144,0.02817893361881,0.02511692885310,0.01449081109889,0.05568061536345,0.05883832648396,0.01989481038031,0.04196863725400,0.02690711151808,0.03258502375446,0.11594915038122,0.11709716543555,0.03367086398143,0.37627973420333,0.39436191320419,0.14785040924578,668.10170777989,GN19529,GN05567/Bjarne
1,145,0.02696298183293,0.02271206118166,0.01296713623498,0.05082092481528,0.04885277338326,0.01682518357852,0.04119368888863,0.02545636519790,0.03052140220178,0.10297616647078,0.10728264600038,0.02296132823749,0.34075735043258,0.36530631780624,0.11106050639497,752.13965844402,Demonstrant,T1005/NK93512
2,146,0.02590350882783,0.02165622822940,0.01452103077283,0.04846454203462,0.04802346602082,0.01992785909139,0.03959594697351,0.02287205122411,0.03296989012317,0.09658321599033,0.10473021864891,0.02962222670910,0.32813140410709,0.32287809252739,0.11978354560703,651.16280834915,GN19524,GN08595/Berserk
3,147,0.02459887203937,0.02048542350531,0.01359734012729,0.04714899867650,0.04710882157087,0.01886670634570,0.03843117157312,0.02204723563045,0.03208380460623,0.09466193198509,0.10316145047545,0.02946936344106,0.31773285328636,0.31554086506367,0.11378397630287,693.31309297913,GN19604,Sommerset/GN08596
4,148,0.02614517630534,0.02118466049433,0.01371351496621,0.04967204578282,0.04753368720412,0.01695254727022,0.03972029382507,0.02270166389644,0.03186389171533,0.10164455316601,0.10553495585918,0.02139195485107,0.34994448360088,0.38023635745049,0.11378944152457,716.29222011385,GN19606,NK01513/Sommerset


### Masbasis_2019

In [ ]:
 Masbasis_2019_x_Field_data_2019

In [ ]:
field = 'Masbasis_2019'
field_plot_id = 'Plot_ID'
ref_df_yield = Masbasis_2019_x_Field_data_2019
red_df_id = 'Rute'

cols2add_dict = {'GrainYield':'GrainYield',
                 'Name':'Name',
#                  'CodeName':'CodeName',
#                  'Pedigree':'pedigree',
                 'Line':'Line',
#                  'Heading_Date':'Hd_date_jd',
#                  'Maturity_Date':'maturity_dates',
                 'Days2Heading':'DH',
                 'Days2Maturity':'DM',
#                  'Lodging':'Lodging_images'
                }


for date in field_year_dict[field]:
    temp_df = locals()[field[:-4]+date].copy()
    temp_ref_df = ref_df_yield.copy()
    
    temp_df.set_index(field_plot_id, inplace=True)
    temp_ref_df.set_index(red_df_id, inplace=True)
    for field_df_col, ref_df_col in cols2add_dict.items():
        temp_df[field_df_col]=temp_ref_df[ref_df_col]
    new_df_name = field[:-4]+date+'_yield'
    locals()[new_df_name] = temp_df.reset_index()
    df_w_yields.append(new_df_name)
    print(new_df_name)
    display(locals()[new_df_name].head())

### Masbasis 2020

In [ ]:
Masbasis_x_20BMLGI1_2020_tm_x_data

In [ ]:
field = 'Masbasis_2020'
field_plot_id = 'Plot_ID'
ref_df_yield = Masbasis_x_20BMLGI1_2020_tm_x_data
red_df_id = 'Rute'

cols2add_dict = {'GrainYield':'GrainYield',
                 'Name':'Name',
#                  'CodeName':'CodeName',
#                  'Pedigree':'pedigree',
                 'Line':'Line',
#                  'Heading_Date':'Hd_date_jd',
                 'Maturity_Date':'maturity_dates',
                 'Days2Heading':'DH',
                 'Days2Maturity':'DM',
                 'Lodging':'Lodging_images'
                }



for date in field_year_dict[field]:
    temp_df = locals()[field[:-4]+date].copy()
    temp_ref_df = ref_df_yield.copy()
    
    temp_df.set_index(field_plot_id, inplace=True)
    temp_ref_df.set_index(red_df_id, inplace=True)
    for field_df_col, ref_df_col in cols2add_dict.items():
        temp_df[field_df_col]=temp_ref_df[ref_df_col]
    new_df_name = field[:-4]+date+'_yield'
    locals()[new_df_name] = temp_df.reset_index()
    df_w_yields.append(new_df_name)
    print(new_df_name)
    display(locals()[new_df_name].head())

### Robot 2020

In [ ]:
Robot_x_ROBOT_2020_x_raw

In [ ]:
%%time

field = 'Robot_2020'
field_plot_id = 'Plot_ID'
ref_df_yield = Robot_x_ROBOT_2020_x_raw
red_df_id = 'SpatialRute'

cols2add_dict = {'GrainYield':'GY_Calc_gm2',
                 'Name':'Name',
                 'CodeName':'CodeName',
#                  'Pedigree':'pedigree',
#                  'Line':'Line',
                 'Heading_Date':'Hd_date_jd',
                 'Maturity_Date':'Mat_date_jd',
                 'Days2Heading':'Hd_dto_day',
                 'Days2Maturity':'Mat_dto_day',
#                  'Lodging':'Lodging_images'
                }



for date in field_year_dict[field]:
    temp_df = locals()[field[:-4]+date].copy()
    temp_ref_df = ref_df_yield.copy()
    
    temp_df.set_index(field_plot_id, inplace=True)
    temp_ref_df.set_index(red_df_id, inplace=True)
    for field_df_col, ref_df_col in cols2add_dict.items():
        temp_df[field_df_col]=temp_ref_df[ref_df_col]
    new_df_name = field[:-4]+date+'_yield'
    locals()[new_df_name] = temp_df.reset_index()
    df_w_yields.append(new_df_name)
    print(new_df_name)
    display(locals()[new_df_name].head())

In [ ]:
# Yield data is required for the following fields/years
sorted_field_year_dict.keys()

In [ ]:
print('Yield data added for the following datasets. ToDo: Yield data for Graminor 2020 and Masbasis 2021 not available.')
df_w_yields

# Create field_year_dict_yield and sorted_field_year_dict_yield since a few datasets have been dropped now

In [ ]:
%%time

field_year_dict_yield = {}

for df in df_w_yields:

    # Splitting the df name to get field name and date
    split_name = df.split('_')
    field_name = split_name[0]
    date1 = split_name[1]
    
    # Creating a dict with all dates for a certain field for a certain year
    dict_key = field_name+'_20'+date1[-2:]
    if dict_key in field_year_dict_yield:
        field_year_dict_yield[dict_key].append(date1)
    else:
        field_year_dict_yield[dict_key] = [date1]
        
#     print(date1)
print('field_year_dict_yield created.')

# pprint(field_year_dict_yield)


In [ ]:
sorted_field_year_dict_yield = {}
for key, dates_list in field_year_dict_yield.items():
    # Converting the dates to a datetime date object and sorting them in list
    sorted_dated = sorted([
        datetime.datetime.strptime(date, '%d%m%y').date()
        for date in dates_list])

    sorted_field_year_dict_yield[key] = sorted_dated
    
print('sorted_field_year_dict_yield created.')
# pprint(sorted_field_year_dict_yield)

# Separate Mean, Median and StdDev datasets with new column names

In [ ]:
%%time
cols2add_dict = {'GrainYield':'GY_Calc_gm2',
                 'Name':'Name',
                 'CodeName':'CodeName',
#                  'Pedigree':'pedigree',
#                  'Line':'Line',
                 'Heading_Date':'Hd_date_jd',
                 'Maturity_Date':'Mat_date_jd',
                 'Days2Heading':'Hd_dto_day',
                 'Days2Maturity':'Mat_dto_day',
#                  'Lodging':'Lodging_images'
                }

general_col_names = ['Plot_ID', 'Blue', 'Green', 'Red', 'RedEdge', 'NIR']
# Can add Days2Heading and Days2Manutiry at later stage as target variables, if required
yield_cols = ['GrainYield', 'Name', 'CodeName', 'Pedigree', 'Line', 'Heading_Date', 'Maturity_Date', 'Days2Heading', 'Days2Maturity', 'Lodging']

df_all_mean = []
df_all_median = []
df_all_stdev = []

for df in df_w_yields:
    temp_mean_df = df+'_mean'
    temp_median_df = df+'_median'
    temp_stdev_df = df+'_stdev'
    
    # Making a temp list of yield columns since all entries from yield cols are not present in every df
    temp_yield_cols = [x for x in locals()[df].columns if x in yield_cols]

    # Filtering columns for each type
    locals()[temp_mean_df] = locals()[df][std_columns_mean+temp_yield_cols]
    locals()[temp_median_df] = locals()[df][std_columns_median+temp_yield_cols]
    locals()[temp_stdev_df] = locals()[df][std_columns_stdev+temp_yield_cols]

    # Renaming column names to general names for all (without mean, median, std_dev in col name)
    locals()[temp_mean_df].columns = general_col_names+temp_yield_cols
    locals()[temp_median_df].columns = general_col_names+temp_yield_cols
    locals()[temp_stdev_df].columns = general_col_names+temp_yield_cols
    

    df_all_mean.append(temp_mean_df)
    df_all_median.append(temp_median_df)
    df_all_stdev.append(temp_stdev_df)

final_df = df_all_mean + df_all_median + df_all_stdev
# final_df
for x in final_df:
    print(x, locals()[x].shape)

In [ ]:
final_df

# Generate more indices

In [ ]:
%%time

# Extract the date from the name of df
date_col = 'Date'
spectral_indices = ['NDVI', 'MTCI', 'DVI', 'GDVI', 'MTCI_CI', 'EXG', 'EXGR', 'RDVI',
                    'TDVI', 'GNDVI', 'NDRE', 'SCCI', 'EVI', 'TVI', 'VARI', 'GARI',
                    'GCI', 'GLI', 'NLI', 'MNLI', 'SAVI', 'GSAVI', 'OSAVI', 'GOSAVI',
                    'MSAVI2', 'MSR', 'GRVI', 'WDRVI', 'SR']
list_df_all_indices = []

for df in final_df:
    new_df_name = df + '_indices'
    date1 = new_df_name.split('_')[1]
    
    temp_df = locals()[df].copy()
    
    # Decided to generate the idices after applying Simpsons integration on the base indices
    
#     ######indices definition
    temp_df['NDVI']= (temp_df['NIR']-temp_df['Red']) / (temp_df['NIR']+temp_df['Red'])
    temp_df['MTCI']= (temp_df['NIR']-temp_df['RedEdge']) / (temp_df['RedEdge']-temp_df['Red'])

    temp_df['DVI']=temp_df['NIR']-temp_df['Red']
    temp_df['GDVI']=temp_df['NIR']-temp_df['Green']
    temp_df['MTCI_CI']=(temp_df['NIR']-temp_df['RedEdge'])/(temp_df['RedEdge']-temp_df['Red'])
    temp_df['EXG']=(2*temp_df['Green'])-temp_df['Red']-temp_df['Blue']
    temp_df['EXGR']=(3*temp_df['Green'])-(2.4*temp_df['Red'])-temp_df['Blue']

    
    temp_df['RDVI']=(temp_df['NIR']-temp_df['Red'])/np.sqrt(temp_df[['NIR','Red']].sum(axis=1))
    temp_df['TDVI']=1.5*(temp_df['NIR']-temp_df['Red'])/np.sqrt((np.power(temp_df['NIR'],2)+ temp_df['Red']+0.5 ))
    temp_df['GNDVI']=(temp_df['NIR']-temp_df['Green'])/(temp_df['NIR']+temp_df['Green'])
    temp_df['NDRE']=(temp_df['NIR']-temp_df['RedEdge'])/(temp_df['NIR']+temp_df['RedEdge'])
    temp_df['SCCI']=temp_df['NDRE']/temp_df['NDVI']
    temp_df['EVI']=2.5*(temp_df['NIR']-temp_df['Red'])/(temp_df['NIR']-6*(temp_df['Red'])-(7.5*temp_df['Blue'])-1)
    temp_df['TVI']=0.5*(120*(temp_df['NIR']-temp_df['Green'])-200*(temp_df['Red']-temp_df['Green']))
    temp_df['VARI']=(temp_df['Green']-temp_df['Red'])/(temp_df['Green']+temp_df['Red']-temp_df['Blue'])
    temp_df['GARI']=(temp_df['NIR' ]-temp_df['Green'])-(1.7*(temp_df['Blue']-temp_df['Red']))/(temp_df['NIR']+temp_df['Green'])-(1.7*(temp_df['Blue']-temp_df['Red']))
    temp_df['GCI']=(temp_df['NIR']/temp_df['Green'])-1
    temp_df['GLI']=(temp_df['Green']-temp_df['Red']-temp_df['Blue'])/(2*temp_df['Green']+temp_df['Red']+temp_df['Blue'])
    temp_df['NLI']=(np.power(temp_df['NIR'],2)-temp_df['Red'])/(np.power(temp_df['NIR'],2)+temp_df['Red'])
    temp_df['MNLI']=(np.power(temp_df['NIR'],2)-temp_df['Red'])*1.5/(np.power(temp_df['NIR'],2)+temp_df['Red'] + 0.5)
    temp_df['SAVI']= ((temp_df['NIR']-temp_df['Red'])*1.5)/(temp_df['NIR']+ temp_df['Red']+ 0.5) 
    temp_df['GSAVI']= ((temp_df['NIR']-temp_df['Green'])*1.5)/(temp_df['NIR']+ temp_df['Green']+ 0.5)                                    
    temp_df['OSAVI']= ((temp_df['NIR']-temp_df['Red']))/(temp_df['NIR']+ temp_df['Red']+ 0.16)
    temp_df['GOSAVI']= ((temp_df['NIR']-temp_df['Green']))/(temp_df['NIR']+ (temp_df['Green'])+ 0.16)
    temp_df['MSAVI2']=(2*temp_df['NIR'])+1-np.sqrt(np.power((2*temp_df['NIR']+1),2)-8*(temp_df['NIR']-temp_df['Red']))/2
    temp_df['MSR']=(temp_df['NIR']/temp_df['Red'])-(1/np.sqrt(temp_df['NIR']/temp_df['Red']))
    temp_df['GRVI']=(temp_df['NIR']/temp_df['Green'])
    temp_df['WDRVI']=((0.1*temp_df['NIR'])-temp_df['Red'])/((0.1*temp_df['NIR'])+temp_df['Red'])
    temp_df['SR']=(temp_df['NIR']/temp_df['Red'])
    
#     # Mistake: Need to define date1 variable
#     temp_df[date_col]=pd.to_datetime(date1, format = '%d%m%y')

    
    list_df_all_indices.append(new_df_name)
    locals()[new_df_name] = temp_df.copy()
list_df_all_indices
for x in list_df_all_indices:
    print(x, locals()[x].shape)

In [ ]:
len(list_df_all_indices)
list_df_all_indices
# Masbasis_040820_mean_indices
print(len(spectral_indices))
Graminor_250719_yield_mean_indices.shape

# Visualisation

## Create Separate df for each index for data plotting

Using median data for further steps

In [36]:
from matplotlib.backends.backend_pdf import PdfPages

# create a PdfPages object
pdf = PdfPages(export_path+'/plots/'+'pllp.pdf')
#     pdf.savefig(fig)

#     # destroy the current figure
#     # saves memory as opposed to create a new figure
#     plt.clf()

# remember to close the object to ensure writing multiple plots


In [37]:
base_indices = ['Blue', 'Green', 'Red', 'RedEdge', 'NIR']
# yield_cols = ['GrainYield']

# data_agg_list = ['_yield_mean_indices', '_yield_median_indices', '_yield_stdev_indices']

# data_agg_list = ['_yield_mean_indices']
data_agg_list = ['_yield_median_indices']
# data_agg_list = ['_yield_stdev_indices']
# base_indices = general_col_names[1:]

# base_indices = ['Blue']
# base_indices = ['Green']
# base_indices = ['Red']
# base_indices = ['RedEdge']
# base_indices = ['NIR']


for d_type in data_agg_list:
    for col in base_indices+spectral_indices:
        print(col)
        fields = len(sorted_field_year_dict_yield.keys())
        rows = math.ceil(fields/2)
        fig_size=(15,10)
        n_plot = 0
        
        if fields < 2:
            columns = 1
            fig, ax = plt.subplots(rows,columns, figsize=fig_size)
            plots = [ax]
        else:
            columns = 2
            fig, ax = plt.subplots(rows,columns, figsize=fig_size)
            plots = ax.flatten()
            
        for field_sample, dates in sorted_field_year_dict_yield.items():
            
            # Adding required data to a temp dataframe
            temp_df = pd.DataFrame()
            for date in dates:
                date_str = date.strftime('%d%m%y')
                field_df = field_sample[:-5]+'_'+date_str+d_type
                col_name = date.strftime('%d-%b')+'_'+col
                temp_df[col_name] = locals()[field_df][col]
            
            # Adding field plot to the subplots
            
            ax_n = plots[n_plot]
            x_labels = [date[:-(len(col)+1)] for date in temp_df.columns.tolist()]
            temp_df.boxplot(ax=ax_n)

            ax_n.set_xticklabels(x_labels, rotation=45)
            data_type = d_type.split('_')[2]
            ax_n.set_title(field_sample+'_'+col+'_'+data_type)
            
#             # Printing the grain yield in plot of the fiels_sample for reference
#             text = "Grain Yield"
#             ax_n.text(0.85, 1.05, text, ha='center', va='top', weight='bold', color='blue', transform=ax_n.transAxes)

            # Create export_path folder if not exists already
            os.makedirs(export_path+'/plots/', exist_ok=True)

            n_plot += 1


        plt.tight_layout()
        pdf.savefig(fig)

#         plt.savefig(export_path+'/plots/'+col+'.png',dpi=500, bbox_inches='tight')
#         plt.savefig(export_path+'/plots/'+col+'.pdf',dpi=500, bbox_inches='tight')
#         plt.savefig(export_path+'/plots/'+col+'.jpg',dpi=150, bbox_inches='tight')

        plt.show()
pdf.close()

NameError: name 'spectral_indices' is not defined

# ToDo: Identify df which are outliers

Identify the datasets from the plots above which do not aligh with the trends of their neighbouring datasets.

## Drop outlier datasets

Graminor 2019	7 Aug Green  
Masbasis 2019	15-Jul BlueGreenRedRedEdgeNIR  
Masbasis 2020	13 jul BlueGreenRedRedEdgeNIR  
Robot 2020	29Jun BlueGreenRedRedEdgeNIR  

In [89]:
outliers = {
    'Graminor_2019':'',
    'Masbasis_2019':'150719',
    'Masbasis_2020':'130720',
    'Robot_2020':'290620'
           }
outliers

{'Masbasis_2019': '150719', 'Masbasis_2020': '130720', 'Robot_2020': '290620'}

## Update field_year_dict and sorted_field_year_dict

In [84]:
sorted_field_year_dict.keys()
# field_year_dict
sorted_field_year_dict_yield.keys()

dict_keys(['Graminor_2019', 'Masbasis_2019', 'Masbasis_2020', 'Robot_2020'])

# Merge data at plots

## Merging datasets on Plot_ID 

In [367]:
# data_agg_list = ['_yield_median_indices']
# yield_cols = ['GrainYield']
list_agg_df = []
# In this loop we select which aggregated data type we will use for further processing.
# It is defined in the list data_agg_list
for d_type in data_agg_list:
    
    for field_sample, dates in sorted_field_year_dict_yield.items():

        # Adding required data to a temp dataframe
        temp_df_all = field_sample+'_all'
        locals()[temp_df_all] = pd.DataFrame()

        for date in dates:
            date_str = date.strftime('%d%m%y')
            field_df = field_sample[:-5]+'_'+date_str+d_type
#             print(field_df)
            temp_df = locals()[field_df].copy()
    
            # Adding date as a suffix to all column headings except Plot_ID
            columns_temp = ['Plot_ID']+[x+'_'+date_str for x in temp_df.columns[1:]]
            
            # Making a temp list of yield columns since all entries from yield cols are not present in every df
            temp_yield_cols = [x for x in temp_df.columns if x in yield_cols]

            # Removing the date suffix from the yield columns
            for ycol in temp_yield_cols:
                temp_yield_col = ycol+'_'+date_str
                if temp_yield_col in columns_temp:
                    index_yield = columns_temp.index(temp_yield_col)
                    if columns_temp[index_yield] == temp_yield_col:
                        columns_temp.remove(temp_yield_col)
                        columns_temp.insert(index_yield, ycol)

            temp_df.columns = columns_temp
            
            # Dropping Date Column, if it exists, as it is not necessary...
            # since date imformation is moved to column headings
            if (date_col+'_'+date_str in temp_df.columns):
                temp_df.drop(columns=[date_col+'_'+date_str], inplace=True)
        
            # For first dataset, keep all columns
            if locals()[temp_df_all].shape[1] < len(base_indices):
                locals()[temp_df_all] = temp_df.copy()
#                 display(temp_df)
            
            # For the remaining, delete yield columns
            else:
                # Dropping grain yield columns to avoid duplication
                for col in temp_yield_cols:
                    temp_df.drop(columns=[col], inplace=True)
                


                # Merging dataframes
#                 locals()[temp_df_all] = pd.concat([locals()[temp_df_all].set_index('Plot_ID'),temp_df.set_index('Plot_ID')], axis=1, join='inner').reset_index()
            
                # The following does not handle duplicate values in the reference column. It will not merge them, rather keep both.
                # So the duplicates should have be dealt with before this step
                locals()[temp_df_all] = locals()[temp_df_all].merge(temp_df, on='Plot_ID', how='left', suffixes=('', date_str))
            
#                 locals()[temp_df_all] = locals()[temp_df_all].join(temp_df, on='Plot_ID', lsuffix='', rsuffix='_right')

#                 display(locals()[temp_df_all])


        list_agg_df.append(temp_df_all)  
        print(temp_df_all, locals()[temp_df_all].shape)
#         display(locals()[temp_df_all].head())
        

Graminor_2019_all (600, 59)
Masbasis_2019_all (528, 46)
Masbasis_2020_all (688, 68)
Robot_2020_all (96, 78)


### Test: Checking if GrainYield columns have been renamed/corrected

In [368]:
# Test: Checking if GrainYield columns have been renamed/corrected

find_text_in_cols = 'Grain'
for df in list_agg_df:
    temp_cols = locals()[df].columns.to_list()
    temp_grain_cols = [s for s in temp_cols if find_text_in_cols in s]
    print(df, temp_grain_cols)

Graminor_2019_all ['GrainYield']
Masbasis_2019_all ['GrainYield']
Masbasis_2020_all ['GrainYield']
Robot_2020_all ['GrainYield']


## Test: Checking if the number of columns is correct

In [369]:
Graminor_050819_yield_median_indices 

,Plot_ID,Blue,Green,Red,RedEdge,NIR,GrainYield,Name,Pedigree
0,101,0.03803116083145,0.08950795233250,0.11108756065369,0.20793855190277,0.33830684423447,499.6244402277,Zebra,Ralle/Dragon
1,102,0.03603747859597,0.08863708749414,0.09795098751783,0.20319047570229,0.33689895272255,515.53275142315,GN14547,SW45126/NK01533
2,103,0.03329745121300,0.08074614033103,0.10181091353297,0.18583717197180,0.31033453345299,529.50102466793,Tarrafal,NaN
3,104,0.03360477834940,0.07923398911953,0.09677043929696,0.18409106135368,0.31025590002537,544.50398481973,GN12760,NK02529/NK01533//NK01568
4,105,0.03652532957494,0.08715170621872,0.10119498148561,0.19789596647024,0.32438720762730,529.50102466793,Bjarne,SvB87293/Bastian
...,...,...,...,...,...,...,...,...,...
595,872,0.04491492733359,0.11676476150751,0.08983241766691,0.21591666340828,0.40492108464241,469.68227703985,GN19602,QUARNA/NK01513//Berserk
596,873,0.04470389150083,0.11402246356010,0.09582930430770,0.21251735836267,0.38515408337116,594.68265654649,GN19587,QUARNA/BAJASS-5//Berserk 4
597,874,0.04412870667875,0.10658716410398,0.09513810649514,0.21512669324875,0.38958498835564,528.06011385199,GN19599,Bombona/NK01513//Berserk
598,875,0.05810645222664,0.13230757415295,0.12883347272873,0.24708507210016,0.43111488223076,489.62914611006,GN19590,Bombona/NK01513//GN03509


In [370]:
# Assert if the number of columns is the same as they should be, based on the number of dates/subsets

for field_sample, dates in sorted_field_year_dict_yield.items():
    dates_satasets = len(sorted_field_year_dict_yield[field_sample])
    temp_all_df = field_sample+'_all'
    print(temp_all_df)
    # Creating a temp list of yield columns present in the subject dataset
    temp_yield_cols = [x for x in locals()[temp_all_df].columns if x in yield_cols]
    assert locals()[temp_all_df].shape[1] == (dates_satasets*len(base_indices))+len(temp_yield_cols)+1

Graminor_2019_all
Masbasis_2019_all
Masbasis_2020_all
Robot_2020_all


## ToDo: Test: Check if the number of subplots/rows is corect

In [371]:
# list_agg_df

# Check subplot with lodging and drop them

In [394]:
# Masbasis 2020, some plots are lodged because of wind and rain. So reflection is not the same
# Remove 70+ lodged lots
# No lodging in 2019
# Mostly in 2020, but arounf 5 plots. Remove them

lodging_col = 'Lodging'
# Lodging threshold, above which all subplots will be dropped
lodging_threshold = 70

lodg_df = []
for df in list_agg_df:
    temp_df = locals()[df]
    if lodging_col in temp_df.columns:
        # Fill the missing values in Lodging column with 1.0
        temp_df['Lodging'] = temp_df['Lodging'].fillna(value=1.0)
        print(df, 'has lodging column.')
        df_lodg = df+'_lodg'
        locals()[df_lodg] = temp_df[temp_df.Lodging < lodging_threshold]
        rows_dropped = temp_df[temp_df.Lodging >= lodging_threshold].shape[0]

        print(f'{rows_dropped} subplots with lodging above the threshold {lodging_threshold} have been droppped.')
        # Appending the name of new df to a list
        lodg_df.append(df_lodg)
        
# Updating the lodg_df list with remaining df wihtout lodging
for df in list_agg_df:
    if lodging_col not in temp_df.columns:
        lodg_df.append(df)
lodg_df

Masbasis_2020_all has lodging column.
29 subplots with lodging above the threshold 70 have been droppped.


['Masbasis_2020_all_lodg',
 'Graminor_2019_all',
 'Masbasis_2019_all',
 'Masbasis_2020_all',
 'Robot_2020_all']

# Exporting the datasets to csv files

In [286]:
os.makedirs(export_path, exist_ok=True)
for df in lodg_df:
    locals()[df].to_csv(export_path+df+'.csv', index=False)

# Test: Missing values in datasets

In [ ]:
print(df_with_nan)
# Finding which column has NAN values
for df in df_with_nan:
    print(f'{df}, {locals()[df].shape[1]-locals()[df].dropna().shape[1]} columns and {locals()[df].shape[0]-locals()[df].dropna().shape[0]} rows to be dropped,')
if len(df_with_nan) > 0:
    raise ValueError

# END OF SECTION

In [ ]:
Graminor_020719.isnull().sum().sort_values

## What is thisssssssss?

In [123]:
# base_indices = ['Blue', 'Green', 'Red', 'RedEdge', 'NIR', 'NDVI', 'MTCI', 'EVI']
# yield_cols = ['GrainYield']

for col in base_indices:
    temp_list = [x for x in Robot_2020_all.columns if (x.find(col)==0) if x != 'MTCI-CI']
    print(len(temp_list), temp_list)

# item.lower().find('sum'.lower()) == -1

NameError: name 'Robot_2020_all' is not defined

# Data Trends

## Normal Distribution of data

ToDo:  
see the distribution of data if it is normal  
else make transpose to make it normal  
dist in Gausion function   
in each field  
what if the data is normal dist?  
the use some transpose to box pox   
try diff funct to see which one iis able to make data normal  
make heat map of whole if not normal  
see which parts are not normal and exculde them  
ls_means in R to make the normalisation/transpose  
pearson corr bw yield and indices for diff dates  


In [ ]:
x_labels

### Yeo-Johnson Transformation

In [ ]:
col_for_plotting = ['Blue', 'Green', 'Red', 'RedEdge', 'NIR', 'NDVI', 'MTCI', 'EVI']

from sklearn.preprocessing import PowerTransformer, normalize, StandardScaler
data_agg_list = ['_median_indices']

# col_for_plotting = ['Blue']
# col_for_plotting = ['Green']
# col_for_plotting = ['Red']

for d_type in data_agg_list:
    for col in col_for_plotting:
        fields = len(field_year_dict_yield.keys())
        rows = math.ceil(fields/2)
        
        fig, ax = plt.subplots(rows,2, figsize=(15,10))
        plots = ax.flatten()
        n = 0
        # TODO: Fix the x ticks
        

        for field_sample, dates in sorted_field_year_dict_yield.items():
            x_labels = []
            # Adding required data to a temp dataframe
            temp_df = pd.DataFrame()
            for date in dates:
                date_str = date.strftime('%d%m%y')
                field_df = field_sample[:-5]+'_'+date_str+d_type
                temp_df[date] = locals()[field_df][col]
                x_label = date.strftime('%d-%m-%y')+':'+str(len(locals()[field_df][col]))
                
                x_labels.append(x_label)
                x_labels= list(set(x_labels))
            # Transform the df
#             pt = PowerTransformer(method='box-cox', standardize=False)
            pt = PowerTransformer(method='yeo-johnson', standardize=False)

            temp_arr = pt.fit_transform(temp_df)
            temp_df = pd.DataFrame(temp_arr)
            
            # Adding field plot to the subplots
            num_of_fields = len(field_year_dict_yield.keys())
            
            text = "Grain Yield"
            ax_n = plots[n]
            
            temp_df.boxplot(ax=ax_n)
            ax_n.set_xticklabels(x_labels, rotation=-35)
            ax_n.set_title(field_sample+'_'+col+d_type[:-5]+'_yeo-johnson')
            
#             # Printing the grain yield in plot of the fiels_sample for reference
#             ax_n.text(0.85, 1.05, text, ha='center', va='top', weight='bold', color='blue', transform=ax_n.transAxes)
            n+=1
        plt.tight_layout()



### Box-Cox Transformation

In [ ]:
col_for_plotting = ['Blue', 'Green', 'Red', 'RedEdge', 'NIR', 'NDVI', 'MTCI', 'EVI']

from sklearn.preprocessing import PowerTransformer, normalize, StandardScaler
data_agg_list = ['_median_indices']

# col_for_plotting = ['Blue']
# col_for_plotting = ['Green']
# col_for_plotting = ['Red']

for d_type in data_agg_list:
    for col in col_for_plotting:
        fields = len(field_year_dict_yield.keys())
        rows = math.ceil(fields/2)
        
        fig, ax = plt.subplots(rows,2, figsize=(15,10))
        plots = ax.flatten()
        n = 0
        # TODO: Fix the x ticks
        for field_sample, dates in sorted_field_year_dict_yield.items():
            
            # Adding required data to a temp dataframe
            temp_df = pd.DataFrame()
            for date in dates:
                date_str = date.strftime('%d%m%y')
                field_df = field_sample[:-5]+'_'+date_str+d_type
                temp_df[date] = locals()[field_df][col]
            x_labels = temp_df.columns.tolist()

            # Transform the df
#             pt = PowerTransformer(method='box-cox', standardize=False)
            pt = PowerTransformer(method='box-cox', standardize=False)

            # Taking absolute values of the dataframe(avoiding negative values)
            temp_arr = pt.fit_transform(temp_df.abs())
            temp_df = pd.DataFrame(temp_arr)
            
            # Adding field plot to the subplots
            num_of_fields = len(field_year_dict_yield.keys())
            
            text = "Grain Yield"
            ax_n = plots[n]

            temp_df.boxplot(ax=ax_n)
            ax_n.set_xticklabels(x_labels, rotation=90)
            ax_n.set_title(field_sample+'_'+col+d_type[:-5]+'_box-cox')
            
#             # Printing the grain yield in plot of the fiels_sample for reference
#             ax_n.text(0.85, 1.05, text, ha='center', va='top', weight='bold', color='blue', transform=ax_n.transAxes)
            n+=1
        plt.tight_layout()



### ToDo: Identify Dates and index with problems

### Ecxclude the problematic data/dates
or
### Take average values where the problematic data is

Take average of data for date 20200708 and 20200624  
Masbasis  
Cleanup  
Remove dates which have drop  

## ToDo: Remove outliers

### Find AUC for all dates of one field
See if it covers tha gaps under the dates,i.e.

Since data points are different  
Flying time is different  
Cover the gaps between the dates  

Since the data collection is not uniform throughout the year so AUC will give a single value instead of multiple values for one field year which will be representative of all the dates 

#### Option 1: Use Scipy

In [ ]:
import scipy
scipy.__version__

In [ ]:
from scipy import integrate
from scipy.integrate import simps

In [ ]:
from scipy.integrate import simpson

In [ ]:
x = np.arange(0, 10)
y = np.arange(0, 10)


In [ ]:
# integrate.simpson(y, x)
integrate.simps(y, x)

In [ ]:
y = np.power(x, 3)
y

In [ ]:
integrate.simpson(y, x)
# integrate.simps(y, x)


In [ ]:
integrate.quad(lambda x: x**3, 0, 9)[0]

In [ ]:
integrate.simpson(y, x, even='first')
# integrate.simps(y, x, even='first')

#### Option 2

In [ ]:
data
# plot: Plot ID
# x: Number of days after sowing or actual date
# y: Value of the index


In [ ]:
# x: Days from sowing to data collection
# May 5 2019 Masbasis and Graminor
# Robot: 

data={'plot':['1','1','2','2','3','3'],'x':['5','6','7','8','9','10'],'y':['0.9','0.8','0.7','0.6','0.5','0.4'] }

ACC=[]
A=pd.DataFrame(data, columns=['plot','x','y'])
AA=0

for item in range(len(A)-1):
    if A['plot'][item]== A['plot'][item+1]:
        Ans=(float((A['y'][item]))+float((A['y'][item+1])))*((float((A['x'][item+1]))-float((A['x'][item]))))/2
        AA+=Ans
        print(AA)
        ACC.append(AA)

### Alternative

In [ ]:
df1=Data.set_index(['Plot'])
ACC=[]

for item in Numbers_final:
    df2=df1[df1.index==item]
    df2=df2.filter(['Blue', 'Green', 'Red', 'RedEdge', 'NIR','NDVI', 'MTCI', 'EVI', 'DVI', 'RVI', 'VARI', 'EXG', 'EXGR', 'GLI', 'GNDVI', 'GVI','Time','timepoint'], axis=1)
    df2=df2.sort_values(by='timepoint')
    df3=df2.reset_index()

AA=0
for j in range(0,3):
    Ans=(float((df3['GVI'][j]))+float((df3['GVI'][j+1])))*((float((df3['timepoint'][j+1]))-float((df3['timepoint'][j]))))/2
    AA+=Ans

    print(AA)
    ACC.append(AA)



DA=pd.DataFrame(ACC)
DD=pd.DataFrame(Numbers_final)
DDA=pd.concat([DD, DA], axis=1)
DDA.to_excel('Staur_Accumulative_GVI_2019.xlsx')

### Time series data vs the AUC

# ToDo: Model Training


Make model for one year at a time and try to predict yield of another field  

TODO: Train on Masbasis 2019 an 2020  
Test on Staur  

Use data until august for yield prediction since it is most relavant  
Use all data for predicting date to maturity  

Data Collection:  
Data collection usually starts after heading  
2019 has the data before hading as well. To use that, dont use dates before heading  

NDVI is resistant to shadows  

DAT390 Report: Do the report with Robot Data only  

TODO: Use AUC for each index for prediction  

TODO:   
Time series data vs the AUC  